In [4]:
import pandas as pd
import yfinance as yf
import talib
from datetime import datetime

# List of all TA-Lib candlestick pattern functions
PATTERNS = {
    'CDL2CROWS': talib.CDL2CROWS,
    'CDL3BLACKCROWS': talib.CDL3BLACKCROWS,
    'CDL3INSIDE': talib.CDL3INSIDE,
    'CDL3LINESTRIKE': talib.CDL3LINESTRIKE,
    'CDL3OUTSIDE': talib.CDL3OUTSIDE,
    'CDL3STARSINSOUTH': talib.CDL3STARSINSOUTH,
    'CDL3WHITESOLDIERS': talib.CDL3WHITESOLDIERS,
    'CDLABANDONEDBABY': talib.CDLABANDONEDBABY,
    'CDLADVANCEBLOCK': talib.CDLADVANCEBLOCK,
    'CDLBELTHOLD': talib.CDLBELTHOLD,
    'CDLBREAKAWAY': talib.CDLBREAKAWAY,
    'CDLCLOSINGMARUBOZU': talib.CDLCLOSINGMARUBOZU,
    'CDLCONCEALBABYSWALL': talib.CDLCONCEALBABYSWALL,
    'CDLCOUNTERATTACK': talib.CDLCOUNTERATTACK,
    'CDLDARKCLOUDCOVER': talib.CDLDARKCLOUDCOVER,
    'CDLDOJI': talib.CDLDOJI,
    'CDLDOJISTAR': talib.CDLDOJISTAR,
    'CDLDRAGONFLYDOJI': talib.CDLDRAGONFLYDOJI,
    'CDLENGULFING': talib.CDLENGULFING,
    'CDLEVENINGDOJISTAR': talib.CDLEVENINGDOJISTAR,
    'CDLEVENINGSTAR': talib.CDLEVENINGSTAR,
    'CDLGAPSIDESIDEWHITE': talib.CDLGAPSIDESIDEWHITE,
    'CDLGRAVESTONEDOJI': talib.CDLGRAVESTONEDOJI,
    'CDLHAMMER': talib.CDLHAMMER,
    'CDLHANGINGMAN': talib.CDLHANGINGMAN,
    'CDLHARAMI': talib.CDLHARAMI,
    'CDLHARAMICROSS': talib.CDLHARAMICROSS,
    'CDLHIGHWAVE': talib.CDLHIGHWAVE,
    'CDLHIKKAKE': talib.CDLHIKKAKE,
    'CDLHIKKAKEMOD': talib.CDLHIKKAKEMOD,
    'CDLHOMINGPIGEON': talib.CDLHOMINGPIGEON,
    'CDLIDENTICAL3CROWS': talib.CDLIDENTICAL3CROWS,
    'CDLINNECK': talib.CDLINNECK,
    'CDLINVERTEDHAMMER': talib.CDLINVERTEDHAMMER,
    'CDLKICKING': talib.CDLKICKING,
    'CDLKICKINGBYLENGTH': talib.CDLKICKINGBYLENGTH,
    'CDLLADDERBOTTOM': talib.CDLLADDERBOTTOM,
    'CDLLONGLEGGEDDOJI': talib.CDLLONGLEGGEDDOJI,
    'CDLLONGLINE': talib.CDLLONGLINE,
    'CDLMARUBOZU': talib.CDLMARUBOZU,
    'CDLMATCHINGLOW': talib.CDLMATCHINGLOW,
    'CDLMATHOLD': talib.CDLMATHOLD,
    'CDLMORNINGDOJISTAR': talib.CDLMORNINGDOJISTAR,
    'CDLMORNINGSTAR': talib.CDLMORNINGSTAR,
    'CDLONNECK': talib.CDLONNECK,
    'CDLPIERCING': talib.CDLPIERCING,
    'CDLRICKSHAWMAN': talib.CDLRICKSHAWMAN,
    'CDLRISEFALL3METHODS': talib.CDLRISEFALL3METHODS,
    'CDLSEPARATINGLINES': talib.CDLSEPARATINGLINES,
    'CDLSHOOTINGSTAR': talib.CDLSHOOTINGSTAR,
    'CDLSHORTLINE': talib.CDLSHORTLINE,
    'CDLSPINNINGTOP': talib.CDLSPINNINGTOP,
    'CDLSTALLEDPATTERN': talib.CDLSTALLEDPATTERN,
    'CDLSTICKSANDWICH': talib.CDLSTICKSANDWICH,
    'CDLTAKURI': talib.CDLTAKURI,
    'CDLTASUKIGAP': talib.CDLTASUKIGAP,
    'CDLTHRUSTING': talib.CDLTHRUSTING,
    'CDLTRISTAR': talib.CDLTRISTAR,
    'CDLUNIQUE3RIVER': talib.CDLUNIQUE3RIVER,
    'CDLUPSIDEGAP2CROWS': talib.CDLUPSIDEGAP2CROWS,
    'CDLXSIDEGAP3METHODS': talib.CDLXSIDEGAP3METHODS
}

# Priority hierarchy: higher absolute value means higher priority
# Negative for bearish, positive for bullish
PATTERN_PRIORITY = {
    # High priority bearish
    'CDLEVENINGSTAR': -10,
    'CDLEVENINGDOJISTAR': -10,
    'CDL3BLACKCROWS': -9,
    'CDLABANDONEDBABY': -9,
    'CDLDARKCLOUDCOVER': -8,
    'CDLENGULFING': -7,  # Bearish version
    # Medium priority bearish
    'CDLBELTHOLD': -6,
    'CDLSHOOTINGSTAR': -5,
    'CDLHANGINGMAN': -5,
    'CDLUPSIDEGAP2CROWS': -4,
    # Low priority bearish
    'CDLDOJI': -3,
    'CDLDOJISTAR': -3,
    'CDLGRAVESTONEDOJI': -2,
    'CDLHIKKAKE': -1,  # Bearish version

    # High priority bullish
    'CDLMORNINGSTAR': 10,
    'CDLMORNINGDOJISTAR': 10,
    'CDL3WHITESOLDIERS': 9,
    'CDLABANDONEDBABY': 9,  # Bullish version
    'CDLPIERCING': 8,
    'CDLENGULFING': 7,  # Bullish version
    # Medium priority bullish
    'CDLHAMMER': 6,
    'CDLINVERTEDHAMMER': 5,
    'CDLDRAGONFLYDOJI': 5,
    'CDLHOMINGPIGEON': 4,
    # Low priority bullish
    'CDLDOJI': 3,
    'CDLDOJISTAR': 3,
    'CDLTAKURI': 2,
    'CDLHIKKAKE': 1,  # Bullish version

    # Add other patterns with appropriate priorities (e.g., -5 to 5)
    'CDL2CROWS': -4,
    'CDL3INSIDE': -3,  # Adjust based on bearish/bullish
    'CDL3LINESTRIKE': -6,
    'CDL3OUTSIDE': -5,
    'CDL3STARSINSOUTH': 4,
    'CDLADVANCEBLOCK': 4,
    'CDLBREAKAWAY': -4,
    'CDLCLOSINGMARUBOZU': -3,
    'CDLCONCEALBABYSWALL': 5,
    'CDLCOUNTERATTACK': -3,
    'CDLGAPSIDESIDEWHITE': -2,
    'CDLHARAMI': 3,
    'CDLHARAMICROSS': 3,
    'CDLHIGHWAVE': -1,
    'CDLHIKKAKEMOD': 2,
    'CDLIDENTICAL3CROWS': -4,
    'CDLINNECK': -2,
    'CDLKICKING': -5,
    'CDLKICKINGBYLENGTH': -5,
    'CDLLADDERBOTTOM': 4,
    'CDLLONGLEGGEDDOJI': -1,
    'CDLLONGLINE': -2,
    'CDLMARUBOZU': -3,
    'CDLMATCHINGLOW': 3,
    'CDLMATHOLD': 4,
    'CDLONNECK': -2,
    'CDLRICKSHAWMAN': -1,
    'CDLRISEFALL3METHODS': 3,
    'CDLSEPARATINGLINES': -3,
    'CDLSHORTLINE': -1,
    'CDLSPINNINGTOP': -1,
    'CDLSTALLEDPATTERN': -3,
    'CDLSTICKSANDWICH': 3,
    'CDLTASUKIGAP': 2,
    'CDLTHRUSTING': -2,
    'CDLTRISTAR': -3,
    'CDLUNIQUE3RIVER': 3,
    'CDLXSIDEGAP3METHODS': -3
}

# Full list of base tickers from user query
BASE_TICKERS = ['BIOCON']

# Create full tickers list with .NS suffix
TICKERS = [ticker + ".NS" for ticker in BASE_TICKERS]

def download_data(ticker, start="2023-01-01", end=None):
    """Download OHLCV data using yfinance."""
    if end is None:
        end = datetime.today().strftime('%Y-%m-%d')
    df = yf.download(ticker, start=start, end=end, interval="5m", progress=False, multi_level_index=False, auto_adjust=True)
    df.dropna(inplace=True)
    return df

def detect_patterns(df):
    """Detect all patterns and return a DataFrame with pattern values."""
    pattern_df = pd.DataFrame(index=df.index)
    for name, func in PATTERNS.items():
        pattern_df[name] = func(df['Open'], df['High'], df['Low'], df['Close'])
    return pattern_df

def prioritize_signals(pattern_df, date=None):
    """Prioritize signals based on pattern strength."""
    if date is None:
        date = pattern_df.index[-1]  # Latest date
    row = pattern_df.loc[date]
    
    signals = []
    for pattern, value in row.items():
        if value != 0:
            priority = PATTERN_PRIORITY.get(pattern, 0)  # Default 0 if not in priority list
            direction = 1 if value > 0 else -1
            score = abs(priority) * direction
            signals.append((pattern, score))
    
    if signals:
        # Sort by absolute score descending (highest priority first)
        signals.sort(key=lambda x: abs(x[1]), reverse=True)
        top_pattern, top_score = signals[0]
        if top_score > 0:
            return f"Bullish {top_pattern} detected - Go Long (Priority Score: {top_score})"
        elif top_score < 0:
            return f"Bearish {top_pattern} detected - Go Short (Priority Score: {top_score})"
    return "No patterns detected or neutral signal"

if __name__ == "__main__":
    # User inputs (optional: adjust start date)
    start_date = "2025-08-20"  # Adjust as needed
    end_date = None  # Today

    # List to hold results
    results = []

    # Loop through all tickers
    for ticker in TICKERS:
        try:
            # Download data
            df = download_data(ticker, start=start_date, end=end_date)
            
            if df.empty:
                results.append({"Ticker": ticker, "Date": "N/A", "Signal": "No data available"})
                continue

            # Detect patterns
            patterns = detect_patterns(df)

            # Get prioritized signal
            priority_signal = prioritize_signals(patterns)

            # Append to results
            results.append({"Ticker": ticker, "Date": str(patterns.index[-1].date()), "Signal": priority_signal})
        except Exception as e:
            results.append({"Ticker": ticker, "Date": "N/A", "Signal": f"Error: {str(e)}"})

    # Create DataFrame and print
    results_df = pd.DataFrame(results)  # Convert results list to DataFrame
    print(results_df.to_string(index=False))
    results_df.to_csv("pattern_signals.csv", index=False)  # Save DataFrame to CSV


   Ticker       Date                                                       Signal
BIOCON.NS 2025-09-05 Bearish CDLBELTHOLD detected - Go Short (Priority Score: -6)


In [15]:
import pandas as pd
import yfinance as yf
import talib
from datetime import datetime, timedelta

# List of all TA-Lib candlestick pattern functions (same as before)
PATTERNS = {
    'CDL2CROWS': talib.CDL2CROWS,
    'CDL3BLACKCROWS': talib.CDL3BLACKCROWS,
    'CDL3INSIDE': talib.CDL3INSIDE,
    'CDL3LINESTRIKE': talib.CDL3LINESTRIKE,
    'CDL3OUTSIDE': talib.CDL3OUTSIDE,
    'CDL3STARSINSOUTH': talib.CDL3STARSINSOUTH,
    'CDL3WHITESOLDIERS': talib.CDL3WHITESOLDIERS,
    'CDLABANDONEDBABY': talib.CDLABANDONEDBABY,
    'CDLADVANCEBLOCK': talib.CDLADVANCEBLOCK,
    'CDLBELTHOLD': talib.CDLBELTHOLD,
    'CDLBREAKAWAY': talib.CDLBREAKAWAY,
    'CDLCLOSINGMARUBOZU': talib.CDLCLOSINGMARUBOZU,
    'CDLCONCEALBABYSWALL': talib.CDLCONCEALBABYSWALL,
    'CDLCOUNTERATTACK': talib.CDLCOUNTERATTACK,
    'CDLDARKCLOUDCOVER': talib.CDLDARKCLOUDCOVER,
    'CDLDOJI': talib.CDLDOJI,
    'CDLDOJISTAR': talib.CDLDOJISTAR,
    'CDLDRAGONFLYDOJI': talib.CDLDRAGONFLYDOJI,
    'CDLENGULFING': talib.CDLENGULFING,
    'CDLEVENINGDOJISTAR': talib.CDLEVENINGDOJISTAR,
    'CDLEVENINGSTAR': talib.CDLEVENINGSTAR,
    'CDLGAPSIDESIDEWHITE': talib.CDLGAPSIDESIDEWHITE,
    'CDLGRAVESTONEDOJI': talib.CDLGRAVESTONEDOJI,
    'CDLHAMMER': talib.CDLHAMMER,
    'CDLHANGINGMAN': talib.CDLHANGINGMAN,
    'CDLHARAMI': talib.CDLHARAMI,
    'CDLHARAMICROSS': talib.CDLHARAMICROSS,
    'CDLHIGHWAVE': talib.CDLHIGHWAVE,
    'CDLHIKKAKE': talib.CDLHIKKAKE,
    'CDLHIKKAKEMOD': talib.CDLHIKKAKEMOD,
    'CDLHOMINGPIGEON': talib.CDLHOMINGPIGEON,
    'CDLIDENTICAL3CROWS': talib.CDLIDENTICAL3CROWS,
    'CDLINNECK': talib.CDLINNECK,
    'CDLINVERTEDHAMMER': talib.CDLINVERTEDHAMMER,
    'CDLKICKING': talib.CDLKICKING,
    'CDLKICKINGBYLENGTH': talib.CDLKICKINGBYLENGTH,
    'CDLLADDERBOTTOM': talib.CDLLADDERBOTTOM,
    'CDLLONGLEGGEDDOJI': talib.CDLLONGLEGGEDDOJI,
    'CDLLONGLINE': talib.CDLLONGLINE,
    'CDLMARUBOZU': talib.CDLMARUBOZU,
    'CDLMATCHINGLOW': talib.CDLMATCHINGLOW,
    'CDLMATHOLD': talib.CDLMATHOLD,
    'CDLMORNINGDOJISTAR': talib.CDLMORNINGDOJISTAR,
    'CDLMORNINGSTAR': talib.CDLMORNINGSTAR,
    'CDLONNECK': talib.CDLONNECK,
    'CDLPIERCING': talib.CDLPIERCING,
    'CDLRICKSHAWMAN': talib.CDLRICKSHAWMAN,
    'CDLRISEFALL3METHODS': talib.CDLRISEFALL3METHODS,
    'CDLSEPARATINGLINES': talib.CDLSEPARATINGLINES,
    'CDLSHOOTINGSTAR': talib.CDLSHOOTINGSTAR,
    'CDLSHORTLINE': talib.CDLSHORTLINE,
    'CDLSPINNINGTOP': talib.CDLSPINNINGTOP,
    'CDLSTALLEDPATTERN': talib.CDLSTALLEDPATTERN,
    'CDLSTICKSANDWICH': talib.CDLSTICKSANDWICH,
    'CDLTAKURI': talib.CDLTAKURI,
    'CDLTASUKIGAP': talib.CDLTASUKIGAP,
    'CDLTHRUSTING': talib.CDLTHRUSTING,
    'CDLTRISTAR': talib.CDLTRISTAR,
    'CDLUNIQUE3RIVER': talib.CDLUNIQUE3RIVER,
    'CDLUPSIDEGAP2CROWS': talib.CDLUPSIDEGAP2CROWS,
    'CDLXSIDEGAP3METHODS': talib.CDLXSIDEGAP3METHODS
}

# Priority hierarchy (same as before)
PATTERN_PRIORITY = {
    'CDLEVENINGSTAR': -10,
    'CDLEVENINGDOJISTAR': -10,
    'CDL3BLACKCROWS': -9,
    'CDLABANDONEDBABY': -9,
    'CDLDARKCLOUDCOVER': -8,
    'CDLENGULFING': -7,
    'CDLBELTHOLD': -6,
    'CDLSHOOTINGSTAR': -5,
    'CDLHANGINGMAN': -5,
    'CDLUPSIDEGAP2CROWS': -4,
    'CDLDOJI': -3,
    'CDLDOJISTAR': -3,
    'CDLGRAVESTONEDOJI': -2,
    'CDLHIKKAKE': -1,
    'CDLMORNINGSTAR': 10,
    'CDLMORNINGDOJISTAR': 10,
    'CDL3WHITESOLDIERS': 9,
    'CDLABANDONEDBABY': 9,
    'CDLPIERCING': 8,
    'CDLENGULFING': 7,
    'CDLHAMMER': 6,
    'CDLINVERTEDHAMMER': 5,
    'CDLDRAGONFLYDOJI': 5,
    'CDLHOMINGPIGEON': 4,
    'CDLDOJI': 3,
    'CDLDOJISTAR': 3,
    'CDLTAKURI': 2,
    'CDLHIKKAKE': 1,
    'CDL2CROWS': -4,
    'CDL3INSIDE': -3,
    'CDL3LINESTRIKE': -6,
    'CDL3OUTSIDE': -5,
    'CDL3STARSINSOUTH': 4,
    'CDLADVANCEBLOCK': 4,
    'CDLBREAKAWAY': -4,
    'CDLCLOSINGMARUBOZU': -3,
    'CDLCONCEALBABYSWALL': 5,
    'CDLCOUNTERATTACK': -3,
    'CDLGAPSIDESIDEWHITE': -2,
    'CDLHARAMI': 3,
    'CDLHARAMICROSS': 3,
    'CDLHIGHWAVE': -1,
    'CDLHIKKAKEMOD': 2,
    'CDLIDENTICAL3CROWS': -4,
    'CDLINNECK': -2,
    'CDLKICKING': -5,
    'CDLKICKINGBYLENGTH': -5,
    'CDLLADDERBOTTOM': 4,
    'CDLLONGLEGGEDDOJI': -1,
    'CDLLONGLINE': -2,
    'CDLMARUBOZU': -3,
    'CDLMATCHINGLOW': 3,
    'CDLMATHOLD': 4,
    'CDLONNECK': -2,
    'CDLRICKSHAWMAN': -1,
    'CDLRISEFALL3METHODS': 3,
    'CDLSEPARATINGLINES': -3,
    'CDLSHORTLINE': -1,
    'CDLSPINNINGTOP': -1,
    'CDLSTALLEDPATTERN': -3,
    'CDLSTICKSANDWICH': 3,
    'CDLTASUKIGAP': 2,
    'CDLTHRUSTING': -2,
    'CDLTRISTAR': -3,
    'CDLUNIQUE3RIVER': 3,
    'CDLXSIDEGAP3METHODS': -3
}

# Tickers (same as before)
BASE_TICKERS = ['ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BEL', 'BHARTIARTL', 'CIPLA', 'COALINDIA', 'DRREDDY', 'EICHERMOT', 'ETERNAL', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'ITC', 'INDUSINDBK', 'INFY', 'JSWSTEEL', 'JIOFIN', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NTPC', 'NESTLEIND', 'ONGC', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SHRIRAMFIN', 'SBIN', 'SUNPHARMA', 'TCS', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'TRENT', 'ULTRACEMCO', 'WIPRO']
TICKERS = [ticker + ".NS" for ticker in BASE_TICKERS]

def download_data(ticker, start="2022-01-01", end=None):
    if end is None:
        end = datetime.today().strftime('%Y-%m-%d')
    df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=True, multi_level_index=False)
    df.dropna(inplace=True)
    return df

def detect_patterns(df):
    pattern_df = pd.DataFrame(index=df.index)
    for name, func in PATTERNS.items():
        pattern_df[name] = func(df['Open'], df['High'], df['Low'], df['Close'])
    return pattern_df

def get_signal(pattern_df, df, date, adx_threshold=25, adx_period=14):
    if date not in pattern_df.index or len(df.loc[:date]) < adx_period:
        return None, None
    
    row = pattern_df.loc[date]
    
    adx = talib.ADX(df['High'][:date], df['Low'][:date], df['Close'][:date], timeperiod=adx_period)[-1]
    
    if adx <= adx_threshold:
        return None, adx
    
    signals = []
    for pattern, value in row.items():
        if value != 0:
            priority = PATTERN_PRIORITY.get(pattern, 0)
            direction = 1 if value > 0 else -1
            score = abs(priority) * direction
            signals.append((pattern, score))
    
    if signals:
        signals.sort(key=lambda x: abs(x[1]), reverse=True)
        top_pattern, top_score = signals[0]
        if top_score > 0:
            return "long", adx
        elif top_score < 0:
            return "short", adx
    return None, adx

def backtest_strategy(df, max_hold_days=10, adx_threshold=25, adx_period=14):
    pattern_df = detect_patterns(df)
    trades = []
    position = None  # 'long' or 'short'
    entry_date = None
    entry_price = None
    
    dates = df.index[adx_period:]  # Start after enough data for ADX
    
    for i, date in enumerate(dates):
        signal, adx = get_signal(pattern_df, df, date, adx_threshold, adx_period)
        
        # Check for exit (opposite signal or max hold)
        if position:
            days_held = (date - entry_date).days
            opposite_signal = (position == 'long' and signal == 'short') or (position == 'short' and signal == 'long')
            if opposite_signal or days_held >= max_hold_days:
                # Exit on next day's open if available
                exit_idx = i + 1
                if exit_idx < len(dates):
                    exit_date = dates[exit_idx]
                    exit_price = df['Open'][exit_date]
                    if position == 'long':
                        pnl = exit_price - entry_price
                    else:  # short
                        pnl = entry_price - exit_price
                    trades.append({
                        'ticker': df.ticker if hasattr(df, 'ticker') else 'Unknown',
                        'entry_date': entry_date,
                        'action': position,
                        'entry_price': entry_price,
                        'exit_date': exit_date,
                        'exit_price': exit_price,
                        'pnl': pnl
                    })
                    position = None
        
        # Check for entry
        if not position and signal:
            # Enter on next day's open if available
            entry_idx = i + 1
            if entry_idx < len(dates):
                entry_date = dates[entry_idx]
                entry_price = df['Open'][entry_date]
                position = signal
    
    # Close any open position at the end
    if position:
        exit_date = df.index[-1]
        exit_price = df['Close'][-1]  # Use close if no next open
        if position == 'long':
            pnl = exit_price - entry_price
        else:
            pnl = entry_price - exit_price
        trades.append({
            'ticker': df.ticker if hasattr(df, 'ticker') else 'Unknown',
            'entry_date': entry_date,
            'action': position,
            'entry_price': entry_price,
            'exit_date': exit_date,
            'exit_price': exit_price,
            'pnl': pnl
        })
    
    return trades

if __name__ == "__main__":
    start_date = "2022-01-01"
    end_date = None  # Today
    max_hold_days = 10
    adx_threshold = 25
    adx_period = 14
    
    all_trades = []
    for ticker in TICKERS[:5]:  # Limit to first 5 tickers for speed; remove slice for all
        try:
            df = download_data(ticker, start=start_date, end=end_date)
            if df.empty:
                continue
            df.ticker = ticker  # Add ticker for reference
            trades = backtest_strategy(df, max_hold_days, adx_threshold, adx_period)
            all_trades.extend(trades)
        except Exception as e:
            print(f"Error for {ticker}: {e}")
    
    # Print trades
    print("Trades Taken:")
    for trade in all_trades:
        print(f"Ticker: {trade['ticker']}, Entry Date: {trade['entry_date'].date()}, Action: {trade['action'].upper()} (Buy for long, Sell for short), Entry Price: {trade['entry_price']:.2f}, Exit Date: {trade['exit_date'].date()}, Exit Price: {trade['exit_price']:.2f}, P&L: {trade['pnl']:.2f}")
    
    # Summary metrics
    if all_trades:
        total_trades = len(all_trades)
        wins = sum(1 for t in all_trades if t['pnl'] > 0)
        win_pct = (wins / total_trades) * 100 if total_trades > 0 else 0
        total_pnl = sum(t['pnl'] for t in all_trades)
        print(f"\nSummary: Total Trades: {total_trades}, Win Percentage: {win_pct:.2f}%, Total P&L: {total_pnl:.2f}")
    else:
        print("No trades generated.")


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_63255/2074742764.py:164: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adx = talib.ADX(df['High'][:date], df['Low'][:date], df['Close'][:date], timeperiod=adx_period)[-1]
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_63255/2074742764.py:235: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exit_price = df['Close'][-1]  # Use close if no next open
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_63255/2074742764.py:164: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always 

Trades Taken:
Ticker: ADANIENT.NS, Entry Date: 2022-01-25, Action: SHORT (Buy for long, Sell for short), Entry Price: 1664.94, Exit Date: 2022-01-27, Exit Price: 1714.84, P&L: -49.91
Ticker: ADANIENT.NS, Entry Date: 2022-01-27, Action: LONG (Buy for long, Sell for short), Entry Price: 1714.84, Exit Date: 2022-01-28, Exit Price: 1699.37, P&L: -15.47
Ticker: ADANIENT.NS, Entry Date: 2022-01-28, Action: SHORT (Buy for long, Sell for short), Entry Price: 1699.37, Exit Date: 2022-02-03, Exit Price: 1768.69, P&L: -69.32
Ticker: ADANIENT.NS, Entry Date: 2022-02-03, Action: LONG (Buy for long, Sell for short), Entry Price: 1768.69, Exit Date: 2022-02-08, Exit Price: 1736.20, P&L: -32.49
Ticker: ADANIENT.NS, Entry Date: 2022-02-08, Action: SHORT (Buy for long, Sell for short), Entry Price: 1736.20, Exit Date: 2022-02-10, Exit Price: 1795.69, P&L: -59.49
Ticker: ADANIENT.NS, Entry Date: 2022-02-10, Action: LONG (Buy for long, Sell for short), Entry Price: 1795.69, Exit Date: 2022-02-22, Exit Pri

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_63255/2074742764.py:164: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adx = talib.ADX(df['High'][:date], df['Low'][:date], df['Close'][:date], timeperiod=adx_period)[-1]
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_63255/2074742764.py:235: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exit_price = df['Close'][-1]  # Use close if no next open
